In [2]:
import requests

# Credenciales de Salesforce
username="jmartinez@careassistance.com"
password="Jcmm010483"
security_token="nc20DsgxOHWVR9OHZ78s7TRp"

# Endpoint de la API de Salesforce
base_url = 'https://careassistance.lightning.force.com'

# Autenticación
session = requests.Session()
session.auth = (username, password + security_token)

# Consulta de ejemplo
query = "SELECT Id, Name FROM Account LIMIT 10"
response = session.get(base_url + f"/services/data/v51.0/query/?q={query}")

# Imprimir resultados
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error al realizar la consulta:", response.content)

TypeError: 'tuple' object is not callable